In [16]:
!git clone https://github.com/vishalpathak24/img-latex.git
%cd img-latex/img-latex

Cloning into 'img-latex'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 19 (delta 1), reused 19 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (19/19), 8.92 KiB | 8.92 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [20]:
!pip install torchserve torch-model-archiver torch-workflow-archiver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 MB 23.7 MB/s eta 0:00:00


In [21]:
!pwd

/content/img-latex


In [36]:
!mkdir ../../sample-images/
!cp /content/drive/MyDrive/Colab\ Notebooks/Image2Latex/sample-images/* ../../sample-images/

mkdir: cannot create directory ‘../../sample-images/’: File exists


In [ ]:
from transformers import VisionEncoderDecoderModel
from transformers.models.nougat import NougatTokenizerFast
from nougat_latex import NougatLaTexProcessor
import torch

# Model Basic Usage

In [ ]:
model_name = "Norm/nougat-latex-base"
device = "cuda" if torch.cuda.is_available() else "cpu"

model = VisionEncoderDecoderModel.from_pretrained(model_name).to(device)

tokenizer = NougatTokenizerFast.from_pretrained(model_name)
latex_processor = NougatLaTexProcessor.from_pretrained(model_name)

In [38]:
from PIL import Image

image = Image.open("../../sample-images/lt-2.jpg")
if not image.mode == "RGB":
    image = image.convert('RGB')

In [39]:
pixel_values = latex_processor(image, return_tensors="pt").pixel_values
decoder_input_ids = tokenizer(tokenizer.bos_token, add_special_tokens=False,
                              return_tensors="pt").input_ids

In [40]:
with torch.no_grad():
    outputs = model.generate(
        pixel_values.to(device),
        decoder_input_ids=decoder_input_ids.to(device),
        max_length=model.decoder.config.max_length,
        early_stopping=True,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        use_cache=True,
        num_beams=5,
        bad_words_ids=[[tokenizer.unk_token_id]],
        return_dict_in_generate=True,
    )

In [41]:
sequence = tokenizer.batch_decode(outputs.sequences)[0]
sequence = sequence.replace(tokenizer.eos_token, "").replace(tokenizer.pad_token, "").replace(tokenizer.bos_token, "")
print(sequence)

\textstyle{\frac{1}{3}}\times\left(6\times{\frac{4}{3}}\right)


# Creating Mar for inferencing

In [43]:
import shutil

shutil.make_archive('nougat', 'zip', 'img-latex/nougat_latex')

'/content/img-latex/img-latex/nougat.zip'

In [42]:
torch.save(model.state_dict(), 'model.pt')

In [44]:
!torch-model-archiver --model-name img-latex \
--version 1.0 --model-file model.py \
--serialized-file model.pt \
--handler handler.py \
--extra-files "nougat.zip"

In [48]:
!mkdir ../../model-store/
!mv img-latex.mar ../../model-store/

mkdir: cannot create directory ‘../../model-store/’: File exists


In [47]:
!torchserve --model-store ../../model-store/ --models img-latex=img-latex.mar &

2024-12-16T12:13:38,056 [INFO ] main org.pytorch.serve.servingsdk.impl.PluginsManager -  Loading snapshot serializer plugin...
nvidia-smi not available or failed: Cannot run program "nvidia-smi": error=2, No such file or directory
2024-12-16T12:13:38,147 [DEBUG] main org.pytorch.serve.util.ConfigManager - xpu-smi not available or failed: Cannot run program "xpu-smi": error=2, No such file or directory
2024-12-16T12:13:38,151 [WARN ] main org.pytorch.serve.util.ConfigManager - Your torchserve instance can access any URL to load models. When deploying to production, make sure to limit the set of allowed_urls in config.properties
2024-12-16T12:13:38,227 [INFO ] main org.pytorch.serve.util.TokenAuthorization - 
######
TorchServe now enforces token authorization by default.
This requires the correct token to be provided when calling an API.
Key file located at /content/img-latex/img-latex/key_file.json
Check token authorization documenation for information: https://github.com/pytorch/serve/

In [49]:
from google.colab import files

files.download('../../model-store/img-latex.mar')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>